# Práctica de Periodismo de Datos: Web Scraping de Titulares y Análisis de Temas

Esta práctica guía a los alumnos a extraer titulares de un sitio de noticias, limpiar el texto y realizar un análisis de frecuencia para identificar las tendencias informativas del día o de una sección específica.

**Objetivos de la Práctica**

1. Web Scraping Estándar: Utilizar requests y BeautifulSoup para extraer titulares y resúmenes de noticias.

2. Limpieza de Texto Avanzada: Procesar el texto eliminando stopwords (palabras comunes) para preparar el análisis.

3. Análisis de Frecuencia: Determinar las palabras clave más recurrentes para identificar los temas de mayor cobertura.

**Requisitos e Instalación**

In [ ]:
pip intall pandas, requests, BeautifulSoup4, re

## OBJETIVO:

1. Web Scraping de Titulares

El alumno debe elegir una sección de un periódico digital pequeño o un blog de noticias que cargue todas las noticias al inicio sin necesidad de scroll dinámico. Esto asegura que requests pueda descargar todo el contenido relevante.

Tarea 1: Extracción de Titulares y Resúmenes
El alumno debe inspeccionar la página para encontrar el contenedor de cada noticia y las etiquetas específicas de los titulares y resúmenes.

Ejemplo: https://canfali.com/

2. Limpieza de Texto y Eliminación de Stopwords

Para identificar los temas, debemos deshacernos de las palabras comunes (stopwords: 'el', 'la', 'un', 'es', 'de') que, aunque son necesarias para la gramática, no nos dicen nada sobre el tema de la noticia.

Tarea 2: Limpiar y Filtrar el Texto

3. Análisis de Frecuencia de Temas

Contando la frecuencia de las palabras clave restantes, podemos cuantificar qué términos son los más importantes en el conjunto de noticias, revelando las tendencias de cobertura informativa.

Tarea 3: Conteo y Análisis de las Palabras Clave